# LAB GenAI - LLMs - OpenAI Assistant

## Exercise 1

Create an assistant to answer a topic of your choosing:
 - Upload a file of your interest
 - Add Instructions to the prompt
 - Use the assistant in Playground mode

 https://platform.openai.com/playground/assistants

In [17]:
pip install openai

  Using cached openai-1.52.2-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.52.2-py3-none-any.whl (386 kB)
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install pymupdf


   ---------------------------------------- 0.0/16.0 MB ? eta -:--:--
   ----- ---------------------------------- 2.4/16.0 MB 13.4 MB/s eta 0:00:02
   ------------- -------------------------- 5.2/16.0 MB 12.8 MB/s eta 0:00:01
   ------------------- -------------------- 7.9/16.0 MB 12.8 MB/s eta 0:00:01
   -------------------------- ------------- 10.5/16.0 MB 12.6 MB/s eta 0:00:01
   -------------------------------- ------- 13.1/16.0 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------  15.7/16.0 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 16.0/16.0 MB 12.4 MB/s eta 0:00:00


In [17]:
import fitz  # PyMuPDF

In [26]:
from openai import OpenAI
client = OpenAI(api_key="")

my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_sQx5cMHMQni98b5bsRngiriI', created_at=1730210146, description=None, instructions='You are an assistant specialized in extracting sustainability reports from user-provided files. Your task is to analyze and summarize key information on a company’s environmental, social, and financial sustainability.\n\nBe precise, critical, and objective in your analysis, presenting the facts clearly and without embellishment. Your goal is to offer an unbiased, straightforward summary, reflecting the true state of the company’s sustainability efforts.', metadata={}, model='gpt-4o', name='Robert', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format=ResponseFormatText(type='text'), temperature=0.44, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=0.66)]


In [34]:
import openai
import fitz  # PyMuPDF

# Initialize the OpenAI client with the API key
client = openai.OpenAI(api_key="")

# Step 1: Create the assistant
assistant = client.beta.assistants.create(
    name="Sustainability Report Extractor",
    instructions="You are an assistant specialized in extracting sustainability reports from user-provided files. Your task is to analyze and summarize key information on a company’s environmental, social, and financial sustainability.",
    model="gpt-3.5-turbo",
    tools=[{"type": "file_search"}]
)

# Step 2: Create a vector store for document storage
vector_store = client.beta.vector_stores.create(name="Sustainability Documents")

# Step 3: Prepare the PDF file for upload
file_paths = ["Group Annual Integrated Report 2023_Generali Group_final_interactive.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

# Step 4: Upload the file to the vector store and poll for completion
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# Check the upload status
print(file_batch.status)
print(file_batch.file_counts)

# Step 5: Link the vector store to the assistant
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

# Step 6: Start a conversation thread to interact with the assistant
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Provide a summary of the company's sustainability report, covering social, environmental, and financial aspects. Filter for things that have actually been done by the company not what is planed future work"
        }
    ]
)

# Run the assistant and get a response
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

# Retrieve and print messages from the thread
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
message_content = messages[0].content[0].text
print(message_content.value)

# Close the file streams after upload
for file in file_streams:
    file.close()


completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)
The sustainability report of Generali Group highlights several key sustainability initiatives in the social, environmental, and financial aspects:

1. **Environmental Sustainability:**
   - Generali is committed to decarbonizing its operations by reducing scope 1, 2, and 3 greenhouse gas (GHG) emissions by at least 35% by 2025 compared to 2019 levels. They are also working towards achieving net-zero status by 2040 through carbon removal projects aligned with reliable protocols and scientific information.
   - The company has made significant progress in reducing GHG emissions from its operations, with a 33.4% reduction in total emissions compared to the 2019 baseline.
   - Generali has increased the share of electricity purchased from renewable sources to 87.1% by 2023, showing a commitment to transitioning to cleaner energy sources.
   - Various initiatives have been implemented to promote sustainability,

In [31]:
print("Assistant ID:", assistant.id)
print("Thread ID:", thread.id)

Assistant ID: asst_9VgKVZXTVAudcTJ3k29wN9gE
Thread ID: thread_DxpkCkk9t0H4FCsaKSuFf3pX


In [32]:
print("Run status:", run.status)
print("Run details:", run)


Run status: failed
Run details: Run(id='run_lZm0syUuQhX2rpOHa174xCiV', assistant_id='asst_9VgKVZXTVAudcTJ3k29wN9gE', cancelled_at=None, completed_at=None, created_at=1730212288, expires_at=None, failed_at=1730212296, incomplete_details=None, instructions='You are an assistant specialized in extracting sustainability reports from user-provided files. Your task is to analyze and summarize key information on a company’s environmental, social, and financial sustainability.', last_error=LastError(code='rate_limit_exceeded', message='Request too large for gpt-4o in organization org-O4bxBuW0QS1D7kcQVJlQhVMy on tokens per min (TPM): Limit 30000, Requested 33195. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.'), max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1730212

In [30]:
# Retrieve and print messages from the thread
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

# Check if the messages list is not empty before accessing it
if messages:
    # Print the full message structure to understand its content if needed
    print("Messages structure:", messages)
    
    # Access the message content safely
    message_content = messages[0].content[0].text if messages[0].content else "No content returned"
    print(message_content)
else:
    print("No messages returned from the assistant.")

# Close the file streams after upload
for file in file_streams:
    file.close()


No messages returned from the assistant.


## Exercise 2

Talk to your assistant via the API

https://platform.openai.com/docs/assistants/overview

In [41]:
import openai

# Initialize your OpenAI client with API key
client = openai.OpenAI(api_key=" ")  # Replace with your actual API key

# Step 1: Start a conversation thread
thread = client.beta.threads.create(
    messages=[{"role": "user", "content": "Hi there! Can you provide a summary of the sustainability report?"}]
)

# Step 2: Run the thread with the assistant
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id="asst_9VgKVZXTVAudcTJ3k29wN9gE"  # Replace with your assistant's ID
)

# Step 3: Retrieve messages from the assistant's response
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

# Step 4: Print the assistant’s response
# Print only the text content from the assistant's response
if messages:
    # Access the message's text content directly
    message_content = messages[0].content[0].text
    print("Assistant's response:", message_content.value)
else:
    print("No response from the assistant.")




Assistant's response: The sustainability report of Generali Group highlights the following key points:

1. **Decarbonization Commitment**:
   - The Group aims to reduce scope 1, 2, and 3 GHG emissions related to its operations by at least 35% by 2025 compared to 2019 levels. The long-term goal is to achieve net-zero status by 2040 through carbon removal projects aligned with reliable protocols and scientific information【4:0†source】.

2. **Renewable Energy**:
   - Generali is committed to converting all its electricity supply contracts to certified renewable energy. In 2023, 87.1% of total purchased electricity was from renewable sources, showing a 5.0 percentage point increase from the baseline【4:0†source】.

3. **Human Safety Net**:
   - The Group's activities aim to unlock the potential of people living in vulnerable conditions through The Human Safety Net, promoting early childhood development and refugee integration. The goal is to impact one million lives by 2027【4:2†source】.

4. *

## Exercise 3

Create an assistant that will call a weather API, given the user's answer and return the proper answer.

See the documentation of the weather API here: https://open-meteo.com/en/docs

In [43]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [46]:
import requests
import openai

# Define the function to fetch weather data
def get_weather_forecast(latitude, longitude):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": "true",
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data["current_weather"]
    else:
        return {"error": "Unable to fetch weather data"}

# Initialize the OpenAI client
client = openai.OpenAI(api_key=" ")  # Replace with your actual API key

# Create a thread to simulate a conversation
thread = client.beta.threads.create(
    messages=[{"role": "user", "content": "What's the current weather in Berlin?"}]
)

# Manually call the weather function based on user input
location = {"latitude": 52.52, "longitude": 13.41}  # Example for Berlin
weather_data = get_weather_forecast(location["latitude"], location["longitude"])

# Format the weather data as a response
if "error" not in weather_data:
    weather_response = (
        f"The current temperature is {weather_data['temperature']}°C with "
        f"{weather_data['windspeed']} km/h winds."
    )
else:
    weather_response = "Sorry, I couldn't fetch the weather data at the moment."

# Print the response
print("Assistant's response:", weather_response)

Assistant's response: The current temperature is 14.8°C with 6.5 km/h winds.


### If you want to, there is a hint here:

OpenAI Chatbots / Assistants have a way to respond in json format. 

Explore the function calling functionality